# center_loss training

paper:  http://ydwen.github.io/papers/WenECCV16.pdf  
code: https://github.com/KaiyangZhou/pytorch-center-loss

## Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
sys.path.append('../')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

### Center-loss algorithm

In [3]:
import torch                                                                                                                                                                                                                                                                 
import torch.nn as nn                                                                                                                                                                                                                                                        

class CenterLoss(nn.Module):                                                                                                                                                                                                                                                 
    """Center loss.                                                                                                                                                                                                                                                          

    Reference:                                                                                                                                                                                                                                                               
    Wen et al. A Discriminative Feature Learning Approach for Deep Face Recognition. ECCV 2016.                                                                                                                                                                              

    Args:                                                                                                                                                                                                                                                                    
     num_classes (int): number of classes.                                                                                                                                                                                                                                
     feat_dim (int): feature dimension.                                                                                                                                                                                                                                   
    """                                                                                                                                                                                                                                                                      
    def __init__(self, num_classes=10, feat_dim=2, use_gpu=True):                                                                                                                                                                                                            
        super(CenterLoss, self).__init__()                                                                                                                                                                                                                                   
        self.num_classes = num_classes                                                                                                                                                                                                                                       
        self.feat_dim = feat_dim                                                                                                                                                                                                                                             
        self.use_gpu = use_gpu                                                                                                                                                                                                                                               

        if self.use_gpu:                                                                                                                                                                                                                                                     
            self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim).cuda())                                                                                                                                                                                 
        else:                                                                                                                                                                                                                                                                
            self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim))                                                                                                                                                                                        

    def forward(self, x, labels):                                                                                                                                                                                                                                            
        """                                                                                                                                                                                                                                                                  
        Args:                                                                                                                                                                                                                                                                
         x: feature matrix with shape (batch_size, feat_dim).                                                                                                                                                                                                             
         labels: ground truth labels with shape (num_classes).                                                                                                                                                                                                            
        """                                                                                                                                                                                                                                                                  
        batch_size = x.size(0)                                                                                                                                                                                                                                               
        distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_classes) + \
            torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, batch_size).t()                                                                                                                                                               
        distmat.addmm_(1, -2, x, self.centers.t())                                                                                                                                                                                                                           

        classes = torch.arange(self.num_classes).long()                                                                                                                                                                                                                      
        if self.use_gpu: classes = classes.cuda()                                                                                                                                                                                                                            
        labels = labels.unsqueeze(1).expand(batch_size, self.num_classes)                                                                                                                                                                                                    
        mask = labels.eq(classes.expand(batch_size, self.num_classes))                                                                                                                                                                                                       

        dist = []                                                                                                                                                                                                                                                            
        for i in range(batch_size):                                                                                                                                                                                                                                          
            value = distmat[i][mask[i]]                                                                                                                                                                                                                                      
            value = value.clamp(min=1e-12, max=1e+12) # for numerical stability                                                                                                                                                                                              
            dist.append(value)                                                                                                                                                                                                                                               
        dist = torch.cat(dist)                                                                                                                                                                                                                                               
        loss = dist.mean()                                                                                                                                                                                                                                                   

        return loss

### Configuration

In [4]:
from sv_system.utils.parser import set_train_config
import easydict
args = easydict.EasyDict(dict(dataset="voxc1_fbank_xvector",
                              input_frames=800, splice_frames=[300, 800], stride_frames=1, input_format='fbank',
                              cuda=True,
                              lrs=[0.1, 0.01], lr_schedule=[20], seed=1337,
                              no_eer=False,
                              batch_size=128,
                              arch="ResNet34", loss="softmax",
                              n_epochs=50
                             ))
config = set_train_config(args)

### Dataset and Dataloader

In [5]:
from sv_system.data.data_utils import find_dataset, find_trial

_, datasets = find_dataset(config, basedir='../')
trial = find_trial(config, basedir='../')

In [6]:
from sv_system.data.dataloader import init_loaders

dataloaders = init_loaders(config, datasets)

### Define Model

In [7]:
# from sv_system.model.model_utils import find_model
# model = find_model(config)

In [8]:
from sv_system.model.ResNet34 import ResNet34

class ResNet34_v1(ResNet34):
    """
        additional fc layer before output layer
    """
    def __init__(self, config, inplanes=16, n_labels=1000, fc_dims=None):
        super().__init__(config, inplanes, n_labels)

        extractor_output_dim = 8*inplanes
        if not fc_dims:
            fc_dims = extractor_output_dim

        classifier = [nn.Linear(extractor_output_dim,
            fc_dims),
            nn.ReLU(inplace=True)]

        loss_type = config["loss"]
        if loss_type == "angular":
            classifier.append(AngleLinear(fc_dims, n_labels))
        elif loss_type == "softmax":
            classifier.append(nn.Linear(fc_dims, n_labels))
        else:
            print("not implemented loss")
            raise NotImplementedError

        self.classifier = nn.Sequential(*classifier)
        
    def forward(self, x):
        feat = self.embed(x)
        out = self.classifier(feat)
        
        return out
    
    def feat_output(self, x):
        feat = self.embed(x)
        out = self.classifier(feat)
        
        return feat, out

In [9]:
model = ResNet34_v1(config, n_labels=config['n_labels'])

if not config['no_cuda']:
    model = model.cuda()

### Model Train

In [10]:
import torch

is_cuda = True
weight_cent = 0.01
criterion_xent = nn.CrossEntropyLoss()
criterion_cent = CenterLoss(num_classes=config['n_labels'], feat_dim=128, use_gpu=is_cuda)
optimizer_model = torch.optim.SGD(model.parameters(), lr=config['lrs'][0], weight_decay=5e-04, momentum=0.9)
optimizer_centloss = torch.optim.SGD(criterion_cent.parameters(), lr=0.5)

In [11]:
from sv_system.train.train_utils import set_seed, find_optimizer

criterion, optimizer = find_optimizer(config, model)

In [12]:
set_seed(config)

In [13]:
if not config['no_eer']:
    train_loader, val_loader, test_loader, sv_loader = dataloaders
else:
    train_loader, val_loader, test_loader = dataloaders

In [14]:
def train(model):
    model.train()
    loss_sum = 0
    xent_loss_sum = 0
    cent_loss_sum = 0
    n_corrects = 0
    total = 0
    for batch_idx, (X, y) in enumerate(train_loader):
        if is_cuda:
            X = X.cuda()
            y = y.cuda()

        feats, outs  = model.feat_output(X)
        loss_xent = criterion_xent(outs, y)
        loss_cent = criterion_cent(feats, y)
        loss_cent *= weight_cent
        loss = loss_xent + loss_cent

        optimizer_model.zero_grad()
        optimizer_centloss.zero_grad()
        loss.backward()
        optimizer_model.step()
        for param in criterion_cent.parameters():                                                                                 
            param.grad.data *= (1. / weight_cent)                                                                            
        optimizer_centloss.step()                                                                                                 
                        
        loss_sum += loss.item()
        xent_loss_sum += loss_xent.item()
        cent_loss_sum += loss_cent.item()
        n_corrects += torch.sum(torch.eq(torch.argmax(outs, dim=1), y)).item()
        total += y.size(0)
        
        if (batch_idx+1) % 100 == 0:
            print("Batch {}/{}\t Loss {:.6f} XentLoss {:.6f} CenterLoss {:.6f}" \
                  .format(batch_idx+1, len(train_loader), loss_sum /(batch_idx+1), 
                        xent_loss_sum/(batch_idx+1), 
                        cent_loss_sum/(batch_idx+1))
                 )
        acc = n_corrects / total

    acc = n_corrects / total
    return loss_sum, acc 

In [15]:
from sv_system.train.si_train import val, sv_test
from tqdm import tqdm_notebook

for epoch_idx in range(0, config['n_epochs']):
    print("-"*30)
    curr_lr = optimizer.state_dict()['param_groups'][0]['lr']
    idx = 0
    while(epoch_idx >= config['lr_schedule'][idx]):
    # use new lr from schedule epoch not a next epoch
        idx += 1
        if idx == len(config['lr_schedule']):
            break
    curr_lr = config['lrs'][idx]
    optimizer.state_dict()['param_groups'][0]['lr'] = curr_lr
    print("curr_lr: {}".format(curr_lr))

#     train code
    train_loss, train_acc = train(model)

#     validation code
    val_loss, val_acc = val(config, val_loader, model, criterion, tqdm=tqdm_notebook)
    print("epoch #{}, val accuracy: {}".format(epoch_idx, val_acc))

#     evaluate best_metric
    if not config['no_eer']:
        # eer validation code
        eer, label, score = sv_test(config, sv_loader, model, trial, tqdm=tqdm_notebook)
        print("epoch #{}, sv eer: {}".format(epoch_idx, eer))

------------------------------
curr_lr: 0.1
Batch 100/1042	 Loss 8.295369 XentLoss 7.084845 CenterLoss 1.210524
Batch 200/1042	 Loss 8.081959 XentLoss 7.013971 CenterLoss 1.067988
Batch 300/1042	 Loss 7.937473 XentLoss 6.974930 CenterLoss 0.962543
Batch 400/1042	 Loss 7.826577 XentLoss 6.948269 CenterLoss 0.878307
Batch 500/1042	 Loss 7.734439 XentLoss 6.926818 CenterLoss 0.807621
Batch 600/1042	 Loss 7.657057 XentLoss 6.909604 CenterLoss 0.747452
Batch 700/1042	 Loss 7.590579 XentLoss 6.894642 CenterLoss 0.695937
Batch 800/1042	 Loss 7.530280 XentLoss 6.879703 CenterLoss 0.650578
Batch 900/1042	 Loss 7.465851 XentLoss 6.854147 CenterLoss 0.611704
Batch 1000/1042	 Loss 7.392661 XentLoss 6.815485 CenterLoss 0.577176



epoch #0, val accuracy: 0.003095518797636032



epoch #0, sv eer: 0.3408050260888084
------------------------------
curr_lr: 0.1
Batch 100/1042	 Loss 6.548639 XentLoss 6.329305 CenterLoss 0.219334
Batch 200/1042	 Loss 6.503753 XentLoss 6.295949 CenterLoss 0.207804
Batch 300/1042	 Loss 6.464415 XentLoss 6.268030 CenterLoss 0.196386
Batch 400/1042	 Loss 6.424708 XentLoss 6.237873 CenterLoss 0.186835
Batch 500/1042	 Loss 6.385294 XentLoss 6.206939 CenterLoss 0.178355
Batch 600/1042	 Loss 6.344319 XentLoss 6.173460 CenterLoss 0.170859
Batch 700/1042	 Loss 6.304958 XentLoss 6.140875 CenterLoss 0.164083
Batch 800/1042	 Loss 6.265977 XentLoss 6.108087 CenterLoss 0.157889
Batch 900/1042	 Loss 6.225379 XentLoss 6.072972 CenterLoss 0.152407
Batch 1000/1042	 Loss 6.183302 XentLoss 6.035934 CenterLoss 0.147368



epoch #1, val accuracy: 0.008402122184634209



epoch #1, sv eer: 0.2595570226812906
------------------------------
curr_lr: 0.1
Batch 100/1042	 Loss 5.624514 XentLoss 5.529137 CenterLoss 0.095377
Batch 200/1042	 Loss 5.565505 XentLoss 5.470582 CenterLoss 0.094922
Batch 300/1042	 Loss 5.494192 XentLoss 5.400556 CenterLoss 0.093636
Batch 400/1042	 Loss 5.422380 XentLoss 5.330403 CenterLoss 0.091978
Batch 500/1042	 Loss 5.360223 XentLoss 5.269383 CenterLoss 0.090840
Batch 600/1042	 Loss 5.293192 XentLoss 5.203536 CenterLoss 0.089656
Batch 700/1042	 Loss 5.231572 XentLoss 5.143157 CenterLoss 0.088415
Batch 800/1042	 Loss 5.167709 XentLoss 5.080266 CenterLoss 0.087443
Batch 900/1042	 Loss 5.105486 XentLoss 5.018936 CenterLoss 0.086549
Batch 1000/1042	 Loss 5.043514 XentLoss 4.957705 CenterLoss 0.085810


...


epoch #47, sv eer: 0.09509104461718666
------------------------------
curr_lr: 0.01
Batch 100/1042	 Loss 0.747557 XentLoss 0.610275 CenterLoss 0.137282
Batch 200/1042	 Loss 0.755258 XentLoss 0.617655 CenterLoss 0.137603
Batch 300/1042	 Loss 0.773295 XentLoss 0.634990 CenterLoss 0.138305
Batch 400/1042	 Loss 0.786485 XentLoss 0.648293 CenterLoss 0.138192
Batch 500/1042	 Loss 0.792103 XentLoss 0.653870 CenterLoss 0.138233
Batch 600/1042	 Loss 0.798877 XentLoss 0.660388 CenterLoss 0.138489
Batch 700/1042	 Loss 0.802361 XentLoss 0.663973 CenterLoss 0.138388
Batch 800/1042	 Loss 0.806796 XentLoss 0.668257 CenterLoss 0.138539
Batch 900/1042	 Loss 0.811466 XentLoss 0.672731 CenterLoss 0.138735
Batch 1000/1042	 Loss 0.815140 XentLoss 0.676345 CenterLoss 0.138795



epoch #48, val accuracy: 0.32382917404174805



epoch #48, sv eer: 0.10222553508678522
------------------------------
curr_lr: 0.01
Batch 100/1042	 Loss 0.745560 XentLoss 0.607308 CenterLoss 0.138251
Batch 200/1042	 Loss 0.741578 XentLoss 0.604557 CenterLoss 0.137022
Batch 300/1042	 Loss 0.765706 XentLoss 0.627556 CenterLoss 0.138150
Batch 400/1042	 Loss 0.776430 XentLoss 0.637941 CenterLoss 0.138489
Batch 500/1042	 Loss 0.781722 XentLoss 0.642833 CenterLoss 0.138889
Batch 600/1042	 Loss 0.790986 XentLoss 0.651945 CenterLoss 0.139041
Batch 700/1042	 Loss 0.797338 XentLoss 0.658374 CenterLoss 0.138964
Batch 800/1042	 Loss 0.802541 XentLoss 0.663565 CenterLoss 0.138975
Batch 900/1042	 Loss 0.807255 XentLoss 0.668208 CenterLoss 0.139048
Batch 1000/1042	 Loss 0.813166 XentLoss 0.674018 CenterLoss 0.139147



epoch #49, val accuracy: 0.3806856572628021



epoch #49, sv eer: 0.09173676924715153
